# Lab 6 Exercises for COMP 6321 (Machine Learning)

In this lab you'll train an AdaBoost classifier and inspect it.

The goal is to get a sense for how a several "weak predictors" can be summed together to make a strong predictor. You will see that the "weak predictors" are in fact simple decision trees (*max_depth=1*), also called "decision stumps" because they are barely trees at all. The use of extremely shallow trees is deliberate: it's their shallowness that makes them 'weak' and yet ripe to be boosted!

**Run the code cell below** to import the required packages.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sklearn.tree        # For DecisionTreeClassifier class
import sklearn.ensemble    # For AdaBoostClassifier class
import sklearn.datasets    # For make_gaussian_quantiles
import sklearn.metrics     # For accuracy_score

<div style="border-bottom: 3px solid black; margin-bottom:5px"></div>
<div style="border-bottom: 3px solid black"></div>

# 1. Understanding how AdaBoost makes predictions

Exercises 1.1&ndash;1.2 ask you to train scikit-learn's AdaBoost classifier (**[sklearn.ensemble.AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)**) and to inspect the way it makes predictions.

**Run the code cell below** to define some useful functions for plotting data and predictions. These are the same functions used for plotting in Lab 5.

In [ ]:
def plot_data(X, y):
    """Plots a toy 2D data set. Assumes values in range [-3,3] and at most 3 classes."""
    plt.plot(X[y==0,0], X[y==0,1], 'ro', markersize=6)
    plt.plot(X[y==1,0], X[y==1,1], 'bs', markersize=6)
    plt.plot(X[y==2,0], X[y==2,1], 'gx', markersize=6, markeredgewidth=2)
    plt.xlim([-3, 3])
    plt.ylim([-3, 3])
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.gca().set_aspect('equal')
    
def plot_predict(model):
    """
    Plots the model's predictions over all points in range 2D [-3, 3].
    If argument is already a Numpy array, treats it as predictions.
    Otherwise calls the argument's predict() function to generate predictions.
    Assumes at most 3 classes.
    """
    extent = (-3, 3, -3, 3)
    x1min, x1max ,x2min, x2max = extent
    x1, x2 = np.meshgrid(np.linspace(x1min, x1max, 100), np.linspace(x2min, x2max, 100))
    X = np.column_stack([x1.ravel(), x2.ravel()])
    y = model.predict(X).reshape(x1.shape)
    cmap = matplotlib.colors.ListedColormap(['r', 'b', 'g'])
    plt.imshow(y, extent=extent, origin='lower', alpha=0.4, vmin=0, vmax=2, cmap=cmap)
    plt.xlim([x1min, x1max])
    plt.ylim([x2min, x2max])
    plt.gca().set_aspect('equal')
    
def plot_class_probability(model, class_index):
    """
    Plots the model's class probability for the given class {0,1,2}
    over all points in range 2D [-3, 3]. Assumes at most 3 classes.
    """
    extent = (-3, 3, -3, 3)
    x1min, x1max ,x2min, x2max = extent
    x1, x2 = np.meshgrid(np.linspace(x1min, x1max, 100), np.linspace(x2min, x2max, 100))
    X = np.column_stack([x1.ravel(), x2.ravel()])
    p = model.predict_proba(X)[:,class_index].reshape(x1.shape)
    colors = [[1, 0, 0], [0, 0, 1], [0, 1, 0]]
    cmap = matplotlib.colors.ListedColormap(np.linspace([1, 1, 1], colors[class_index], 50))
    plt.imshow(p, extent=extent, origin='lower', alpha=0.4, vmin=0, vmax=1, cmap=cmap)
    plt.xlim([x1min, x1max])
    plt.ylim([x2min, x2max])
    plt.gca().set_aspect('equal')    

<div style="border-bottom: 3px solid black;"></div>

### Exercise 1.1 &mdash; Compare an AdaBoost classifier to a simple decision tree

Start by training a decision tree like you did in Lab5 so that we can compare the decision tree to an AdaBoost classifier in Exercise 1.2.

**Repeat exercise 1.1 from Lab5.** Specifically, you are asked to

1. Build the small 2D training set below:

$$
X = \begin{bmatrix}
-1 & 0\\
-\frac{1}{3} & 0\\
\frac{1}{3} & 0\\
 1 & 0
\end{bmatrix},
\quad
y = \begin{bmatrix}
0\\
1\\
0\\
1
\end{bmatrix}
$$

2. Train a decision tree classifier on $X$ and $y$. Use argument *random_state*=0.
3. Plot the decision tree predictions and the data (use *plot_predict* and *plot_data* from preamble).
4. Plot the decision tree itself (use **[sklearn.tree.plot_tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html)**); pass `feature_names=['x1', 'x2']` as an argument.

*Note:* If you receive an error that `sklearn.tree.plot_tree` does not exist, it means you are not using a recent enough version of *sklearn*. If you are on a lab machine booted to Windows, you must reboot to Linux.

In [ ]:
# Your code here. Aim for 8-12 lines.
X = np.array([[-1, 0],
              [-1/3, 0],
              [1/3, 0],
              [ 1, 0]])
y = np.array([0, 1, 0, 1])

tree = sklearn.tree.DecisionTreeClassifier(random_state=0)
tree.fit(X, y)

plt.figure()
plot_predict(tree)
plot_data(X, y)

plt.figure()
sklearn.tree.plot_tree(tree, feature_names=['x1', 'x2']);

By now you should be able to understand exactly what you see above: how the decision tree thresholds and the leaves determine the red/blue regions in feature space. You know from lecture that the training accuracy of a decision tree classifier is improved by "splitting" a leaf.

AdaBoost does not try to improve training accuracy by adding complexity to the current tree current. In fact, each tree that AdaBoost builds only has one split (*max_depth*=1), so it is hardly a tree at all. Such a 'weak' tree is often called a "decision stump" instead of "decision tree."

Here you will train an AdaBoost classifier and see how it combines these 'stumps' to give a result comparable to the decision tree you trained. The decision stumps of an AdaBoostClassifier object are available via the *estimators_* attribute, which is a list of DecisionTreeClassifier objects.

**Write a few lines of code** to:
1. Train an **[AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)** on the same data set. Use *n_estimators*=3 as an argument when building your classifier in order to keep it simple enough to inspect closely. Use *algorithm*='SAMME' to use a more classical variant of AdaBoost that more closely aligns with what you will learn in class. Use *random_state*=0 for reproducibility.
2. Plot the decision regions and data like you did for the decision tree.
3. Plot the tree structure of each decision stump in a separate figure. Use *sklearn.tree.plot_tree*.

In [ ]:
# Your code here. Aim for 4-6 lines.
boosted_trees = sklearn.ensemble.AdaBoostClassifier(n_estimators=3, random_state=0, algorithm='SAMME')
boosted_trees.fit(X, y)

plt.figure()
plot_predict(boosted_trees)
plot_data(X, y)

Now you have now trained an AdaBoost classifier and should see that it generates the same predictions as your original decision tree.

Finally, you should inspect the individual decision 'stumps' that AdaBoost is using to generate its predictions. You should generate three figures (one for each stump, of which there are *n_estimators*). Each figure should have two subplots: on the left plot decision regions of the stump along with the training data, and on the right plot the split that the stump uses to predict class labels. Your first figure should like this:

![image](img/adaboost-simple-stump-0.png)

Each decision stump (each estimator) has a weight associated with it. This is the weight the model is given when making predictions. Find the weight using the *estimator_weights_* attribute of the AdaBoostClassifier and include the weight as part of the title, as shown above.

**Write plotting code** below.

In [ ]:
# Your code here. Aim for 8-10 lines.
for i in range(boosted_trees.n_estimators):
    fig = plt.figure(figsize=(8,4))
    plt.subplot(1, 2, 1)
    plot_predict(boosted_trees.estimators_[i])
    plot_data(X, y)
    plt.title("estimator[%d] (weight %.3f)" % (i, boosted_trees.estimator_weights_[i]))
    
    plt.subplot(1, 2, 2)
    sklearn.tree.plot_tree(boosted_trees.estimators_[i], feature_names=['x1', 'x2']);

The AdaBoost training procedure that led to these particular stumps will be made clear in lecture, but is not important for this lab. The rough idea behind AdaBoost training is that the $R^\text{th}$ decision stump is trained to fix the mistakes (misclassifications) that the previous decision stumps $1,\ldots,R-1$ are currently making on the training set. As long as this new decision stump can fix at least one mistake, we can continue "boosting" by adding more.

<div style="border-bottom: 3px solid black;"></div>

### Exercise 1.2 &mdash; Compute an AdaBoost prediction "by hand"

Notice in Exercise 1.1 that complex decision regions can be built not just by deepening a decision tree, but also by somehow combining the predictions of decision stumps. Here you will learn how that 'combining' happens, by reproducing AdaBoost's predictions from the decision stumps using your own code.

**Run the code cell below** to define a function that will plot the training data along the $x$-axis only, using just component $x_{i1}$ of each training point $\mathbf{x}_i = (x_{i1}, x_{i2})$. For this exercise your plots will use the $y$-axis to represent the probability of class 1 (blue).

In [ ]:
def plot_data_1d(X, y):
    """
    Plots just the first component of a toy 2D data set.
    Assumes values in range [-3,3] and at most 3 classes.
    """
    plt.plot([-3, 3], [0.5, 0.5], '--k', alpha=0.25)
    plt.plot(X[y==0,0], X[y==0,1]*0+0.5, 'ro', markersize=6)
    plt.plot(X[y==1,0], X[y==1,1]*0+0.5, 'bs', markersize=6)
    plt.plot(X[y==2,0], X[y==2,1]*0+0.5, 'gx', markersize=6, markeredgewidth=2)
    plt.xlim([-3, 3])
    plt.xlabel('x1')

First, plot each decision stump's class prediction along the $x_1$ dimension, using the $y$-axis to show the class probability. You should generate three figures, one for each decision stump (each estimator). Your first figure should look like this:
![image](img/adaboost-simple-stump-0-one-dimension.png)
Note that the green line here is not a decision boundary! We are plotting a 1-dimensional feature space, so the place where the green line crosses the $y$-axis at 0.5 is the decision boundary! The above plot corresponds to the decision region of the first decision stump you plotted in Exercise 1.1.

**Write plotting code** to show the separate class prediction of each stump. Use a for-loop to generate a new figure for each decision stump. Use the [np.linspace](https://numpy.org/devdocs/reference/generated/numpy.linspace.html) function to generate a range of $x_1$ values in $[-3, 3]$. Since your AdaBoost predictor from Exercise 1.1 expects a feature matrix $X$ of shape $(N,2)$, you will need to fill the second column with zeros in order to successfully call the classifier's *predict* function. Once you have the vector of class predictions, use a single call to Matplotlib's *plot* function to plot all a class predictions for all $x_1$ values. 

In [ ]:
# Your code here. Aim for 10-12 lines.
N = 500
x1vals = np.linspace(-3, 3, N)
x2vals = np.zeros(N)
Xvals = np.column_stack([x1vals, x2vals])

for i in range(boosted_trees.n_estimators):
    yvals = boosted_trees.estimators_[i].predict(Xvals)
    plt.figure()
    plt.plot(x1vals, yvals, ':g')
    plot_data_1d(X, y)
    plt.title('stump %d' % i)
    plt.ylabel('stump prediction')
    plt.ylim(-0.05, 1.05)

You should see that the class predictions correspond with the red/blue decision regions for each stump that you observed at the end of Exercise 1.1.

Now, rather than plotting the individual class predictions of each stump, you will plot a *weighted combination* of these class predictions. The specific weighting scheme describe here is the one that AdaBoost uses to make predictions. Plot the following function over $\mathbf{x}=(x_1, x_2)$ where $x_1 \in [-3,3]$ and $x_2=0$:

$$
y(\mathbf{x}) = \frac{\sum_{r=1}^R w_r f_r(\mathbf{x})}{\sum_{r=1}^R w_r} 
$$

where $f_r(\mathbf{x})$ is the class prediction of the $r^\text{th}$ decision stump in the AdaBoost classifier and $w_r$ is its weight. You should know how to compute the above using Numpy and the *estimators_* and *estimator_weights_* attributes of the trained AdaBoostClassifier object. 

For example, if you made an AdaBoost classifier with $R=2$ decision stumps, this is what your plot would look something like this:

![image](img/adaboost-simple-weighted-stumps-1.png)

Generate a similar plot but for the case of **three decision stumps**, not two as shown above.

**Write code** to create a weighted combination of the decision stump class predictions as described above, and plot the resulting class scores.

In [ ]:
# Your code for weighting the predictions here. Aim for 2-8 lines.
weights = boosted_trees.estimator_weights_
stumps = boosted_trees.estimators_
weighted_predict = sum(w * stump.predict(Xvals) for w, stump in zip(weights, stumps))
weighted_predict /= weights.sum()

# Your plotting code here. Aim for 5-6 lines.
plt.figure()
plt.plot(x1vals, weighted_predict, ':g')
plot_data_1d(X, y)
plt.title("weighed combination of class predictions");
plt.ylabel('weighted stump predictions')
plt.ylim(-0.05, 1.05);

Finally, compare your plot above to the result of calling *decision_function* on your trained AdaBoostClassifier object. However, the scikit-learn AdaBoostClassifier object treats binary classification special: it generates class predictions in range $[-1, 1]$ rather than in range $[0,1]$. So, you may have to scale (by factor of $\frac{1}{2}$) and shift (by $+\frac{1}{2}) the AdaBoostClassifier's decision function.

**Write a few lines of code** to call *decision_function* to generate predictions over the range $x_1 \in [-3, 3]$ just as you did already. (You can re-use variables that you already defined from earlier code cells.) The plot you generate using scikit-learn should be identical to the one you generated by hand, and it should correctly classify all red and blue points in the training set.

In [ ]:
# Your code here. Aim for 6-7 lines.
plt.figure()
plt.plot(x1vals, boosted_trees.decision_function(Xvals)*0.5+0.5, ':g')
plot_data_1d(X, y)
plt.title("(decision_function+1)/2")
plt.ylabel('weighted stump predictions')
plt.ylim(-0.05, 1.05);

<div style="border-bottom: 3px solid black; margin-bottom:5px"></div>
<div style="border-bottom: 3px solid black"></div>

# 2. Plotting how AdaBoost increases the training accuracy

Exercises 2.1&ndash;2.2 ask you to train and inspect an AdaBoost classifier on a real data set, to see how each successive 'weak' classifier increases the overall training accuracy when weighted with the previous weak classifiers.

<div style="border-bottom: 3px solid black;"></div>

### Exercise 2.1 &mdash; Create a classification dataset using scikit-learn

Here you'll create a synthetic dataset using one of scikit-learn's dataset utilities. Use the **[sklearn.datasets.make_gaussian_quantiles](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_gaussian_quantiles.html)** function to generate a 2-dimensional synthetic dataset with three classes (the default).
 
**Write a few lines of code** to generate the dataset and plot it. Use *random_state*=0. Your plot should look something like this:
![image](img/gaussian-quantiles-data.png)


In [ ]:
# Your code here. Aim for 6-8 lines.
X, y = sklearn.datasets.make_gaussian_quantiles(n_samples=100, random_state=0)

plt.scatter(X[y==0,0], X[y==0,1], s=30, c='r', marker='x')
plt.scatter(X[y==1,0], X[y==1,1], s=30, c='b', marker='x')
plt.scatter(X[y==2,0], X[y==2,1], s=30, c='g', marker='x')
plt.gca().set_aspect('equal')
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plt.title("synthetic data");

<div style="border-bottom: 3px solid black;"></div>

### Exercise 2.2 &mdash; Train an AdaBoost classifier on the Iris dataset and plot the decision regions

You must train an AdaBoost classifier on the synthetic training set from Exercise 2.1. Your plot should look something like this:
![image](img/gaussian-quantiles-adaboost-1.png)

**Write a few lines of code** train the classifier and plot the resulting decision regions. Use the *plot_predict* and *plot_data* functions defined at the start of the lab. Use *random_state*=0 and *algorithm*='SAMME' as before.

In [ ]:
# Your code here. Aim for 4-6 lines.
model = sklearn.ensemble.AdaBoostClassifier(random_state=0)
model.fit(X, y)
    
plot_predict(model)
plot_data(X, y)
plt.title('AdaBoost decision regions');

You may notice that the default AdaBoost hyperparameters struggle to fit this particular data. You can try playing with the *learning_rate* or *n_estimators* parameters to see how they effect the result.

<div style="border-bottom: 3px solid black;"></div>

### Exercise 2.3 &mdash; Plot the decision regions of successive boosting "rounds"

You are asked to plot the decision regions of an AdaBoost classifier with *n_estimators*$=1, 2, \ldots, 10$ decision stumps (estimators). Your first plot should look like this:

![image](img/gaussian-quantiles-adaboost-stages-1.png)

**Write a few lines of code** to generate the plots. Use a for-loop in your solution and for each plot you should display the *accuracy_score* on the training set.

In [ ]:
# Your code here. Aim for 7-10 lines.
for R in range(1, 11):
    model = sklearn.ensemble.AdaBoostClassifier(n_estimators=R, algorithm="SAMME", random_state=0)
    model.fit(X, y)

    accuracy = sklearn.metrics.accuracy_score(y, model.predict(X))
    
    plt.figure()
    plot_predict(model)
    plot_data(X, y)
    plt.title("%d stumps (%.1f%% accuracy)" % (R, accuracy*100))    

<div style="border-bottom: 3px solid black;"></div>

### Exercise 2.4 &mdash; Plot the training accuracy of successive boosting "rounds"

You are asked to plot the accuracy of an AdaBoost classifier with *n_estimators*$=1, 2, \ldots, 100$ decision stumps (estimators). This is just like Exercise 2.3 except you do not plot the decision regions, and instead keep a record of all the accuracies. Your plot should end up looking like this:

![image](img/gaussian-quantiles-adaboost-accuracy.png)

If your accuracy seems to plateau at a much lower value than depicted above, try switching *algorithm* to 'SAMME.R' (the default in scikit-learn).

**Write a few lines of code** to generate the plot. Once you have a list of accuracies, plot them with a single call to Matplotlib's *plot* function.

In [ ]:
# Your code here. Aim for 9-12 lines.
accuracies = []
for R in range(1, 101):
    model = sklearn.ensemble.AdaBoostClassifier(n_estimators=R, random_state=0)
    model.fit(X, y)
    accuracies.append(100*sklearn.metrics.accuracy_score(y, model.predict(X)))

plt.plot(range(1, 101), accuracies, '-k')
plt.title('training accuracy of successive boosting rounds')
plt.xlabel('$R$')
plt.ylabel('percent accuracy');
plt.ylim(0, 100);